In [ ]:
using Plots, PyPlot, DynamicalSystems, Random, Distributions, StatsBase, DataFrames, CSV, Dates, KernelDensity, Interpolations, StatsPlots


In [ ]:
# data = CSV.File(string("../../Advanced_Analytics/Dissertation/Code/MDTG-MALABM/Data/CoinTossX/L1LOBStar.csv"), drop = [:MidPrice, :Spread, :Price], missingstring = "missing", types = Dict(:DateTime => DateTime, :Initialization => Symbol, :Type => Symbol)) |> x -> filter(y -> y.Initialization != :INITIAL, x) |> DataFrame

date = DateTime("2019-07-08")
startTime = date + Hour(9) + Minute(1)
endTime = date + Hour(16) + Minute(50) 

data = CSV.File(string("../../Advanced_Analytics/Dissertation/Code/MDTG-MALABM/Data/JSE/L1LOB.csv"), drop = [:MidPrice, :Spread, :Price], missingstring = "missing", types = Dict(:DateTime => DateTime, :Type => Symbol)) |> DataFrame
filter!(x -> startTime <= x.DateTime && x.DateTime < endTime, data)

data.Date = Date.(data.DateTime)
uniqueDays = unique(data.Date)
uniqueDays = unique(data.Date)
logreturns = map(day -> diff(log.(skipmissing(data[searchsorted(data.Date, day), :MicroPrice]))), uniqueDays) |> x -> reduce(vcat, x)


In [ ]:
# bin the log returns for smoothing
function bin(logreturns, bins)
    logreturns_binned = Vector{Float64}()
    for i in bins:length(logreturns)
        start = (i-bins)+1
        push!(logreturns_binned, mean(logreturns[start:i]))
    end
    return logreturns_binned
end

In [ ]:
# remove the transient
transient = 100
logreturns_trns = logreturns[transient:end]

In [ ]:
# bin the log returns with no transient
logreturns_binned = bin(logreturns_trns, 10)

In [ ]:
# plot the binned log returns
plt.plot(logreturns_binned)


## Compute the time delay parameter

In [ ]:
# set parameters
lags = 1:100

In [ ]:
# compute the time delay using the autocorrelation of the micro-prices (first min since min is always 1)
return_autocor = autocor(logreturns_binned, lags)
plt.plot(return_autocor)
xlabel("lags")
ylabel("Autocor")
tau = estimate_delay(logreturns_binned, "ac_min", lags)
println("Tau: " * string(tau))


## Compute the correlation dimension (do box-counting for sanity)
Note: D2 can't exceed 2log10(N), where N is the number of data points

In [ ]:
Dmax = 2 * log10(length(logreturns_binned))

In [ ]:
# compute the GP correlation dimension and use m to compute the required delay dimension
function PlotCorrelationVsBoxSize(data, dims, es_starts, es_stops, es_step, tau)
    
    slopes = Vector{Float64}()
    
    subplot_dims = Int(ceil(length(dims)/3))
#     fig = plt.figure(figsize=plt.figaspect(0.5))

    for (i, dim) in enumerate(dims)
        # set box size range
        es = ℯ .^ (es_starts[i]:es_step:es_stops[i])
        es_log = log.(es)
        
        # create correlation embeddings
        recon = embed(data, dim, tau)
        cs = correlationsum(recon, es; q = 2)
        reg_res = linear_region(log.(es), log.(cs); tol = 0.8)
        slope = reg_res[2]
        push!(slopes, slope)
        
        # make plots
        plt.plot(es_log, cs, label = dim)
        plt.plot(es_log[reg_res[1][1]:reg_res[1][2]], cs[reg_res[1][1]:reg_res[1][2]])
    end
    plt.legend()
    
    return slopes
    
end

In [ ]:
dims = [1,2,5,10,15,20,25,30,35,40,45] 
es_starts = [-13.5,-13,-11,-10.5,-10.5,-10,-10,-9.8,-9.7,-9.5,-9.3]
es_stops = ones(length(dims)) .* -7.5
es_step = 0.1 
tau = 10
slopes = PlotCorrelationVsBoxSize(logreturns_binned, dims, es_starts, es_stops, es_step, tau)

In [ ]:
plt.plot(dims, slopes, marker = "o")
xlabel("Embedding dimension")
ylabel("Fractal dimension")


A quick note on hyperparams:
1. Tau: If tau <= 5 then we have the the above analysis suggests the embedding dimension should be 1, however if tau > 5, (at least less than 20) we have that the embedding dim should be 2. All experiments suggest convergence and constant behaviour after about 30 embedding dimensions.
2. Increasing the binning increases tau and decreases the number of dimensions.

Also interesting to note that I tested the JSE and the fractal dimension evened out around 5 or 6.

In [ ]:
# create the necessary embedding tau = firstmin(autocorrelation), dim = ceil(max_fractal)
tau = 10
dim = 2
R = embed(logreturns_binned, 2, 10)

In [ ]:
# perform any dim reduction needed to create a visualisation of the embedding

In [ ]:
# plot embedding
start = 1
finish = length(R)
plt.plot(R[start:finish, 1], R[start:finish, 2], lw = 1, marker = "o", ms = 0.1, alpha = 0.5)
plt.title("Returns")

## Smooth the embedding using NN

In [ ]:
# can do ball or square, I will do ball first because it is easiest
function GetNeighbours(data, point, nlast, epsilon)
    neighboursX = Vector{Float64}()
    neighboursY = Vector{Float64}()
    neighboursT = Vector{Int64}()
    neigboursDists = Vector{Float64}()
    for i in 1:size(data)[1]
        # compute dist to all other point
        next_point = data[i,:]
        if next_point.t == point.t # dont compare to itself
            continue
        else
            dist = sqrt((point.x - next_point.x)^2 + (point.y - next_point.y)^2)
            if dist <= epsilon && (point.t - nlast) <= next_point.t && next_point.t < point.t # check if in ball and check if in time bound
                push!(neighboursX, next_point.x)
                push!(neighboursY, next_point.y)
                push!(neighboursT, next_point.t)
                push!(neigboursDists, dist)
            end
        end
    end
    if length(neighboursX) == 0
        neighboursX = point.x
        neighboursY = point.y
        neighboursT = point.t
        neigboursDists = 0
    end
    return neighboursX, neighboursY, neighboursT, neigboursDists
end

In [ ]:
t,(x,y) = collect(1:length(R)), columns(R)
embedded_dataset = DataFrame(Dict("t"=>t,"x"=>x,"y"=>y))

In [ ]:
nlast = 30
epsilon = 1e-3
GetNeighbours(embedded_dataset, embedded_dataset[1,:], nlast, epsilon)

In [ ]:
function NearestNeighboursSmoothing(data, nlast, epsilon)
    smoothed_vecs = Vector{Vector{Float64}}()
    for i in 1:size(data)[1]
        neighboursX, neighboursY, neighboursT, neigboursDists = GetNeighbours(data, data[i,:], nlast, epsilon)
#         mean(neighboursX[i] * 1/(neigboursDists[i]) for i in 1:length(neighboursT)), mean(neighboursY[i] * 1/(neigboursDists[i]) for i in 1:length(neighboursT))
        if i % 100 == 0
            println("I: ", i," Number of neighbours: ", length(neighboursX), " Average dist: ", mean(neigboursDists))
        end
        push!(smoothed_vecs, [mean(neighboursX), mean(neighboursY)])
    end
    return smoothed_vecs
end

In [ ]:
smoothed_vecs = NearestNeighboursSmoothing(embedded_dataset, nlast, epsilon)

In [ ]:
# perform required dim reduction

In [ ]:
# plot smoothed embedding
start = 1
finish = length(smoothed_vecs)
plt.plot([s for (s,j) in smoothed_vecs], [j for (s,j) in smoothed_vecs], lw = 1, marker = "o", ms = 0.1, alpha = 0.5)
plt.title("Returns")


In [ ]:
x = [s for (s,j) in smoothed_vecs]
xdot = [j for (s,j) in smoothed_vecs]
# perm = sortperm(x)
# x_sort = x[perm]
# xdot_sort = xdot[perm]
# k = kde((x_sort, xdot_sort))
# ik = InterpKDE(k)
# z = pdf(ik, x_sort, xdot_sort)


In [ ]:
Plots.plot(x, xdot, linewidth = 1, alpha = 0.5)
Plots.histogram2d!(x, xdot, show_empty=false, bins = 150, c = cgrad(:winter, [0, 0.1, 0.50, 1]))


In [ ]:
density = kde((x, xdot))
Plots.plot(x, xdot, linewidth = 1, alpha = 0.5)
StatsPlots.plot!(density, levels = 1500, c = cgrad(:winter, [0, 0.1, 0.50, 1]))